In [18]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np

client = OpenAI(
    api_key=os.environ.get('OPENAI_API_KEY'),  # This is the default and can be omitted
)

from kaggle_submission import SubmissionBase

questions = pd.read_csv('test.csv')


**V1 - Simple Context Improvement**



In [7]:
context = 'You are a reliability professor. You will see a multiple choice question. Please output one or several correct answer. You should only return the letter of the correct answer. For example, if you want to say that answer [d] is the right one, you should only retun "d". Only one response is admissible'

context = """
You are an AI expert in reliability engineering. Your task is to answer multiple-choice questions (MCQs) accurately and concisely. Each question will have exactly one correct answer.

Instructions:
- Read the question and the possible answers.
- Identify the single correct answer based on your expertise in reliability engineering.
- Respond only with the letter of the correct answer (e.g., a, b, c, or d). Do not provide any explanations or additional text.
For example, if you want to say that answer [d] is the right one, you should only retun "d".


Example usage:
Question: Which metric measures the average time between system failures?
a. MTTR
b. MTBF
c. Availability
d. Failure Rate

Expected response:
b
"""

def sendRequestToGPT(prompt,
                     context="You are a reliability professor. You will see a multiple choice question. Please output one or several correct answer.",
                     print_=False):
    try:
        # Send a chat completion request to GPT-4
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": context},
                {"role": "user", "content": prompt}
            ]
        )
        
        # Retrieve and print the response text
        response_text = response.choices[0].message.content

        if print_:
            print('Context:')
            print(context)
            print('')
            print('Prompt:')
            print(prompt)
            print('')
            print('GPT-4 Response:')
            print(response_text)
        return response_text
    except Exception as e:
        print(f"An error occurred: {e}")

class SimpleContext(SubmissionBase):
    def get_1_answer(self, q):
        return sendRequestToGPT(q,context)
    
v1 = SimpleContext(questions)
v1.submission_to_csv('submission1.csv')
v1.score()

 --> Prediction 1, question 1 : a
 --> Prediction 1, question 2 : a
 --> Prediction 1, question 3 : c
 --> Prediction 1, question 4 : a
 --> Prediction 1, question 5 : a
 --> Prediction 1, question 6 : b
 --> Prediction 1, question 7 : d
 --> Prediction 1, question 8 : a
 --> Prediction 1, question 9 : b
 --> Prediction 1, question 10 : a
 --> Prediction 1, question 11 : c
 --> Prediction 1, question 12 : a
 --> Prediction 1, question 13 : a
 --> Prediction 1, question 14 : d
 --> Prediction 1, question 15 : a
 --> Prediction 1, question 16 : c
 --> Prediction 1, question 17 : b
 --> Prediction 1, question 18 : b
 --> Prediction 1, question 19 : a
 --> Prediction 1, question 20 : c
 --> Prediction 1, question 21 : c
 --> Prediction 1, question 22 : a
 --> Prediction 1, question 23 : a
 --> Prediction 1, question 24 : d
 --> Prediction 2, question 1 : c
 --> Prediction 2, question 2 : a
 --> Prediction 2, question 3 : c
 --> Prediction 2, question 4 : b
 --> Prediction 2, question 5 : a

0.4833333333333333

**Double Prompting**


In [2]:
from pydantic import BaseModel
from typing import Literal

class FullReasoning(BaseModel):
    steps: list[str]
    final_answer: Literal["a", "b", "c", "d"]

SYSTEM_PROMPT = """
You are a reliability expert. Respond with a, b, c, or d.
"""

DOUBT_PROMPT = """
I have a doubt. Are you totally sure ? Double-check your answer and explain briefly in 2 steps.
"""

class DoublePrompting(SubmissionBase):
    def get_1_answer(self, q):

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": q},
        ]

        first_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.61,
            max_completion_tokens=50,
        )

        messages += [
            {"role": "assistant", "content": first_response.choices[0].message.content},
            {"role": "user", "content": DOUBT_PROMPT},
        ]

        response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.61,
            #max_completion_tokens=300,
            response_format=FullReasoning,
        )
        answer = response.choices[0].message.parsed
        return answer.final_answer

In [5]:
double_prompting = DoublePrompting(questions)
double_prompting.submission_to_csv('submission.csv')
double_prompting.score()

Mean accuracy: 0.2666666666666667


0.2666666666666667

**Multiway prompting**

In [ ]:
from kaggle_submission import SequentialQuestions
class MultiPrompting(SequentialQuestions):
    def get_1_answer(self, q, return_log=False):
        
        context = """You are a reliability expert. You will be asked to answer to several questions based on your knowledge and the definitions you know.
You will need to explain your reasonning and explain the steps that allowed you to choose your answers."""

        def extract_question(q):
            split = q.split('\n')
            return split[0],split[2]
        question,choices = extract_question(q)

        questions = [
            question,
            f"""Based on your previous answer, you should now assess the veracity of each of the following possible answers one by one:
{choices}""",
            f"""Now please select the 1 possibility that fits bests the initial question.
It is possible that none of the possible answer seems acceptable to you. In this case, please choose the one that is the closest to your opinion.
Please note that only one answer is acceptable and that only the letter of the correct answer is expected. If you want to say that answer [d] is the right one, you should only retun "d"."""
        ]

        conversation = self._ask_questions_in_a_row(context, questions)

        if return_log:
            return conversation
        return list(conversation[-1].values())[0]
     

mp = MultiPrompting(questions)
q = """[Question]: 2. In general, reliability testing is performed for which of the following reasons?I. To detect unanticipated failure modes.II. To compare estimated failure rates to actual failure rates.III. To monitor reliability growth over time.IV. To meet or exceed customer expectations.

[Choices]: [a] I and III only | [b] II and IV only | [c] I, II and III only  | [d] I, II, III and IV"""
mp.get_1_answer(q, True)

[('0-system',
  'You are a reliability expert. You will be asked to answer to several questions based on your knowledge and the definitions you know.\nYou will need to explain your reasonning and explain the steps that allowed you to choose your answers.'),
 ('1-user',
  '[Question]: 2. In general, reliability testing is performed for which of the following reasons?I. To detect unanticipated failure modes.II. To compare estimated failure rates to actual failure rates.III. To monitor reliability growth over time.IV. To meet or exceed customer expectations.'),
 ('2-assistant',
  "Reliability testing is a critical component in the development and validation of products and systems to ensure they perform as expected over their intended lifecycle. Let's analyze each of the provided options to determine their relevance to the reasons for conducting reliability testing.\n\nI. **To detect unanticipated failure modes.**\n   - **Reasoning:** One of the primary objectives of reliability testing i

In [3]:
mp = MultiPrompting(questions)
df_test = mp.test(n=2)
display(df_test)

1
('0-system', 'You are a reliability expert. You will be asked to answer to several questions based on your knowledge and the definitions you know.\nYou will need to explain your reasonning and explain the steps that allowed you to choose your answers.')
('1-user', '[Question]: 2. In general, reliability testing is performed for which of the following reasons?I. To detect unanticipated failure modes.II. To compare estimated failure rates to actual failure rates.III. To monitor reliability growth over time.IV. To meet or exceed customer expectations.')
('2-assistant', "Reliability testing is an essential part of the product development process aimed at ensuring that a product will perform its intended function without failure over a specified period. Let's analyze the provided options one by one:\n\nI. **To detect unanticipated failure modes**: \nThis is indeed a primary reason for reliability testing. Reliability testing can help uncover potential weak points or unforeseen failure mod

,question_id,question,answer,0-system,1-user,2-assistant,3-user,4-assistant,5-user,6-assistant
0,1,"[Question]: 2. In general, reliability testing...",d,You are a reliability expert. You will be aske...,"[Question]: 2. In general, reliability testing...",Reliability testing is an essential part of th...,"Based on your previous answer, you should now ...",Let's evaluate each choice in light of the pre...,Now please select the 1 possibility that fits ...,d
1,2,"[Question]: 7. Five items were placed on test,...",c,You are a reliability expert. You will be aske...,"[Question]: 7. Five items were placed on test,...",To find the 95% lower confidence limit for the...,"Based on your previous answer, you should now ...",To assess the veracity of each possible answer...,Now please select the 1 possibility that fits ...,d


In [ ]:
#mp.submission_to_csv('submission_multi_discussion.csv')
#mp.score()

**RAG prompting**

1. Construct the knowledge database
2. Build functions to get cosine similarity with OpenAI embedding
3. Make Sequencial Prompting with the appropriate context, additional information, question and followed questions

In [19]:
KNOWLEDGE = [
    """First piece of knowledge.""",
    """Second piece of knowledge.""",
]

In [11]:
def cosine_similarity(vec1:np.array, vec2:np.array)->float:
    """
    Computes the cosine similarity between two vectors.

    Cosine similarity measures the cosine of the angle between two non-zero vectors in an inner product space. 
    It is a value between -1 and 1, where:
    - 1 indicates that the vectors are identical in direction,
    - 0 indicates that the vectors are orthogonal,
    - -1 indicates that the vectors are diametrically opposed.

    The formula for cosine similarity is:
        cosine_similarity = dot(vec1, vec2) / (||vec1|| * ||vec2||)
    where:
        - dot(vec1, vec2) is the dot product of vec1 and vec2,
        - ||vec1|| is the Euclidean norm (magnitude) of vec1,
        - ||vec2|| is the Euclidean norm (magnitude) of vec2.

    Parameters:
    ----------
    vec1 : np.ndarray
        A 1-dimensional NumPy array representing the first vector. It must be non-empty.
    vec2 : np.ndarray
        A 1-dimensional NumPy array representing the second vector. It must have the same dimensions as `vec1`.

    Returns:
    -------
    float
        The cosine similarity value between `vec1` and `vec2`.

    Raises:
    ------
    ValueError
        If `vec1` and `vec2` are not 1-dimensional arrays or have different lengths.
    ZeroDivisionError
        If either `vec1` or `vec2` is a zero vector (i.e., all elements are zero).

    Examples:
    --------
    >>> vec1 = np.array([1, 2, 3])
    >>> vec2 = np.array([4, 5, 6])
    >>> cosine_similarity(vec1, vec2)
    0.9746318461970762

    >>> vec1 = np.array([1, 0, 0])
    >>> vec2 = np.array([0, 1, 0])
    >>> cosine_similarity(vec1, vec2)
    0.0
    """
    # Check if vectors are of the same size
    if vec1.shape != vec2.shape:
        raise ValueError("Both vectors must have the same dimensions.")
    
    # Compute the dot product
    dot_product = np.dot(vec1, vec2)
    
    # Compute the magnitudes of the vectors
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    # Check for zero vectors to avoid division by zero
    if norm_vec1 == 0 or norm_vec2 == 0:
        raise ZeroDivisionError("One or both of the vectors is a zero vector.")

    # Return the cosine similarity
    return dot_product / (norm_vec1 * norm_vec2)

# Example usage
vector1 = np.array([1, 2, 3])
vector2 = np.array([4, 5, 6])
print(f"Cosine similarity: {cosine_similarity(vector1, vector2)}")


Cosine similarity: 0.9746318461970762


In [23]:
def get_embedding(text:str)->np.array:
    """
    Retrieves the embedding of a given text using a pre-defined OpenAI client.
    
    Parameters:
        text (str): The input string to embed.
    
    Returns:
        list: A list of floats representing the embedding.
    """
    response = client.embeddings.create(
        input=text,
        model='text-embedding-3-small',
        encoding_format='float',
    )
    list_ = response.data[0].embedding
    return np.array(list_)
    
# Example usage
text = "OpenAI provides state-of-the-art AI models."
embedding = get_embedding(text)

# Print the embedding vector
print(f"Embedding for the text:\n{text}\n")
print(embedding[:10])
print(f"Embedding length: {len(embedding)}")


Embedding for the text:
OpenAI provides state-of-the-art AI models.

[-0.02089412 -0.02022826  0.03428014 -0.0039607   0.0133975  -0.04220153
 -0.04766615  0.03065237 -0.00910387 -0.00090551]
Embedding length: 1536
